In [ ]:
from mt import *

In [ ]:
# The directory containing the images
data_dir = '/Volumes/martin/A08_A09_test'

# glob string for images
im_glob = os.path.join(data_dir, 'Round1_max_composite.tif')

# Get list of images
im_list = sorted(glob.glob(im_glob))

im_list

In [ ]:
# Read data using skimage
im = skimage.io.imread(im_list[0])

In [ ]:
i = 25
af488 = im[i][0]
af561 = im[i][1]
af594 = im[i][2]
af633 = im[i][3]
cfp = im[i][4]
dapi = im[i][5]

In [ ]:
af561 = register_images(af488, af561)
af594 = register_images(af488, af594)
af633 = register_images(af488, af633)

In [ ]:
af488 = imadjust(af488)
af561 = imadjust(af561)
af594 = imadjust(af594)
af633 = imadjust(af633)

In [ ]:
af488_g, af561_g, af594_g = filt_gauss(af488, af561, af594, 1.5)
af633_g, cfp_g, dapi_g = filt_gauss(af633, cfp, dapi, 1.5)

In [ ]:
threshold = threshold_otsu(dapi_g)
print(threshold, 'is where the cutoff point is.')
dapi_bw = dapi_g > threshold

In [ ]:
dapi_labels, df_dapi, dapi_area, dapi_distance, dapi_markers = ws(dapi_bw, 2, np.ones((45,45)), 2000, dapi)

In [ ]:
bokeh.io.show(show_four_ims(dapi, dapi_distance, dapi_markers, dapi_labels,
                             cmap=[None, colorcet.gray, colorcet.gray, colorcet.b_glasbey_hv]))

Next I will apply a LoG filter. This filter detects edges, which are defined by areas of sudden peaks or valleys in the gradient (first derivative) of the pixel intensity values. A peak or a valley in the first derivative means there is a zero-crossing in the second derivative.

In [ ]:
af488_edge_zero, af488_LoG, af488_edge = log_filter(af488_g, 1.5, 4, 0.9)
af561_edge_zero, af561_LoG, af561_edge = log_filter(af561_g, 1.5, 4, 0.9)
af594_edge_zero, af594_LoG, af594_edge = log_filter(af594_g, 1.5, 4, 0.9)
af633_edge_zero, af633_LoG, af633_edge = log_filter(af633_g, 1.5, 4, 0.9)

In [ ]:
# Show result
bokeh.io.show(show_four_ims(af488_g, af488_LoG, af488_edge, af488_edge_zero,
                            titles=["488", "LoG", "edge", "zero"]))

In [ ]:
# Show result
bokeh.io.show(show_four_ims(af488_g, af488_edge_zero, af594_g, af594_edge_zero,
                            titles=["488", "488", "594", "594"]))

In [ ]:
# Show result
bokeh.io.show(show_four_ims(af594_g, af594_edge_zero, af633_g, af633_edge_zero,
                            titles=["561", "561", "633", "633"]))

## Step 2. Segmentation

Next, I will skeletonize to get single pixel edges.

In [ ]:
# Skeletonize edges
af488_edge_sk = skimage.morphology.skeletonize(af488_edge_zero)
af561_edge_sk = skimage.morphology.skeletonize(af561_edge_zero)
af594_edge_sk = skimage.morphology.skeletonize(af594_edge_zero)
af633_edge_sk = skimage.morphology.skeletonize(af633_edge_zero)

# See result
bokeh.io.show(show_two_ims(af488_g, af488_edge_sk, titles=["original", "edges"]))

I will fill the holes created from the edges. Note that this will not fill holes that have any openings in them, even if the openings are very small.

In [ ]:
# Fill holes
af488_bw = ndi.morphology.binary_fill_holes(af488_edge_sk)
af561_bw = ndi.morphology.binary_fill_holes(af561_edge_sk)
af594_bw = ndi.morphology.binary_fill_holes(af594_edge_sk)
af633_bw = ndi.morphology.binary_fill_holes(af633_edge_sk)

In [ ]:
# Show result
bokeh.io.show(show_four_ims(af488_g, af488_bw, af594_g, af594_bw,
                            titles=["488", "488", "594", "594"]))

I will remove small objects. Note that this removes any objects that are not filled as well and will remove dots if they are not already filled.

In [ ]:
# Remove small objectes that are not bacteria
af488_bw = skimage.morphology.remove_small_objects(af488_bw, min_size=10)
af561_bw = skimage.morphology.remove_small_objects(af561_bw, min_size=10)
af594_bw = skimage.morphology.remove_small_objects(af594_bw, min_size=10)
af633_bw = skimage.morphology.remove_small_objects(af633_bw, min_size=10)

In [ ]:
af488_labels, df_af488, af488_area, af488_distance, af488_markers = ws(af488_bw, 1, np.ones((8,8)), 10, af488)
af561_labels, df_af561, af561_area, af561_distance, af561_markers = ws(af561_bw, 1, np.ones((8,8)), 10, af561)
af594_labels, df_af594, af594_area, af594_distance, af594_markers = ws(af594_bw, 1, np.ones((8,8)), 10, af594)
af633_labels, df_af633, af633_area, af633_distance, af633_markers = ws(af633_bw, 1, np.ones((8,8)), 10, af633)

In [ ]:
bokeh.io.show(show_four_ims(af488_g, af488_labels, af594_g, af594_labels,
                            titles=["488", "488", "594", "594"],
                            cmap=[None, colorcet.b_glasbey_hv, None, colorcet.b_glasbey_hv]))

In [ ]:
selem = skimage.morphology.disk(1)
af488_e = skimage.morphology.erosion(af488_labels, selem)
af594_e = skimage.morphology.erosion(af594_labels, selem)

In [ ]:
bokeh.io.show(show_six_ims(af488_g, af488_labels, af488_e, af594_g, af594_labels, af594_e,
                            titles=["488", "488", "488", "594", "594", "594"],
                            cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv,
                                  None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
af488_594 = np.logical_or(af488_e, af594_e)

bokeh.io.show(show_five_ims(af488_e, af594_e, af488_594, af488_g, af594_g,
                             titles=['488', '594', 'OR', '488', '594'], 
                             cmap=[colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, None, None, None]))